
<p><img src="https://oceanprotocol.com/static/media/banner-ocean-03@2x.b7272597.png" alt="drawing" width="800" align="center"/>


<h1><center>Ocean Protocol - Manta Ray project</center></h1>
<h3><center>Decentralized Data Science and Engineering, powered by Ocean Protocol</center></h3>
<p>Version 0.4 - alpha preview</p>
<p><a href="https://github.com/oceanprotocol/mantaray">mantaray on Github</a></p>
<p>

# Getting Underway - wallets, passwords and tokens

To interact in Ocean Protocol, you will need an account, which you will fund with Token to access the assets
in the network.

In this notebook, we will demonstrate this behaviour with pre-loaded accounts.

To use Ocean, a User requires
- A user account address
- A password
- Ocean Token

### Section 0: Import modules, and setup logging

In [4]:
# Standard imports
import logging
import random
from pprint import pprint

# Import mantaray and the Ocean API (squid)
# mantaray_utilities is an extra helper library to simulate interactions with the Ocean API.
import squid_py
from squid_py.ocean.ocean import Ocean
from squid_py.config import Config
import mantaray_utilities as manta_utils
logging.info("Squid API version: {}".format(squid_py.__version__))
# Setup logging to a higher level and not flood the console with debug messages
# manta_utils.logging.logger.setLevel('CRITICAL')
manta_utils.logging.logger.setLevel('INFO')

20 - <ipython-input-4-9e73a995bba1> - <module>        - Squid API version: 0.4.3


In [5]:
# Get the configuration file path for this environment
# You can specify your own configuration file at any time, and pass it to the Ocean class.
# os.environ['USE_K8S_CLUSTER'] = 'true'
logging.critical("Deployment type: {}".format(manta_utils.config.get_deployment_type()))
CONFIG_INI_PATH = manta_utils.config.get_config_file_path()
logging.critical("Configuration file selected: {}".format(CONFIG_INI_PATH))

50 - <ipython-input-5-49b527de6115> - <module>        - Deployment type: USE_K8S_CLUSTER
50 - <ipython-input-5-49b527de6115> - <module>        - Configuration file selected: /home/batman/ocn/mantaray_jupyter/config_k8s_deployed.ini


## Section 1: Examine the configuration object

In [6]:
# The API can be configured with a file or a dictionary.
# In this case, we will instantiate from file, which you may also inspect.
# The configuration is a standard library [configparser.ConfigParser()](https://docs.python.org/3/library/configparser.html) object.
print("Configuration file:", CONFIG_INI_PATH)
configuration = Config(CONFIG_INI_PATH)
pprint(configuration._sections)

Configuration file: /home/batman/ocn/mantaray_jupyter/config_k8s_deployed.ini
OrderedDict([('keeper-contracts',
              OrderedDict([('keeper.url', 'https://nile.dev-ocean.com'),
                           ('keeper.path', 'artifacts_nile'),
                           ('gas_limit', '1000000'),
                           ('secret_store.url',
                            'https://secret-store.dev-ocean.com'),
                           ('parity.url', 'https://nile.dev-ocean.com'),
                           ('parity.address',
                            '0x413c9ba0a05b8a600899b41b0c62dd661e689354'),
                           ('parity.password', 'ocean_secret'),
                           ('parity.address2',
                            '0x064789569D09b4d40b54383d84A25A840E5D67aD'),
                           ('parity.password2', 'ocean_secret')])),
             ('resources',
              OrderedDict([('aquarius.url',
                            'https://nginx-aquarius.dev-ocean.com/

Let's look at the 2 parameters that define your identity
The 20-byte 'parity.address' defines your account address
'parity.password' is used to decrypt your private key and securely sign transactions

In [7]:
user1_address = configuration['keeper-contracts']['parity.address']
user1_pass = configuration['keeper-contracts']['parity.password']
print("Currently selected address:", user1_address)
print("Associated password:", user1_pass)

Currently selected address: 0x413c9ba0a05b8a600899b41b0c62dd661e689354
Associated password: ocean_secret


Alternatively, for the purposes of these demos, a list of passwords for local and cloud testing are available.
Several utility functions have been created to manage these passwords for testing multiple users.

In [8]:
# Load the passwords file
path_passwords = manta_utils.config.get_project_path() / 'passwords.csv'
passwords = manta_utils.user.load_passwords(path_passwords)
user1_pass = manta_utils.user.password_map(user1_address, passwords)

20 - user            - load_passwords  - 31 account-password pairs loaded


## Section 2: Instantiate the Ocean API class with this configuration
The Ocean API has an attribute listing all created (simulated) accounts in your local node

In [9]:
ocn = Ocean(configuration)
logging.critical("Ocean smart contract node connected ".format())

20 - ocean_secret_store - __init__        - 	SecretStore: url https://secret-store.dev-ocean.com, parity-client https://nile.dev-ocean.com, account 0x413c9ba0a05b8a600899b41b0c62dd661e689354
20 - diagnostics     - verify_contracts - Keeper contract artifacts (JSON abi files) at: /home/batman/ocn/mantaray_jupyter/artifacts_nile
20 - diagnostics     - verify_contracts - Using keeper contracts from network nile, network id is 8995
20 - diagnostics     - verify_contracts - Looking for keeper contracts ending with ".nile.json", e.g. "ServiceExecutionAgreement.nile.json".
20 - diagnostics     - verify_contracts - Finished loading keeper contracts:
	Dispenser: 0xb8B0ec3AC0bf28ebB47b3cce4b1B7607DD7FA2DB
	OceanToken: 0x88CAA68F41DD7cFdD431BcA036E11bd20ef58882
	DIDRegistry: 0x9d306Ca587ff4b311C7963e62F48f3d6B59Ec1a1
	ServiceExecutionAgreement: 0xFfCb6bea15BbF19Dd3bCDC82f1864A92F359284A
	PaymentConditions: 0xc00b256Ff109EDaA5A375799CFB7386221863329
	AccessConditions: 0x1be580a31D79a7FAcF1F5c70d8F

An account has a balance of Ocean Token, Ethereum, and requires a password to sign any transactions.

In [10]:
# List the accounts in the network
print(len(ocn.accounts.list()), "accounts exist")

# Print a simple table listing accounts and balances
print("{:<5} {:<45} {:<20} {:<12} {}".format("","Address", "Ocean Token Balance", "Password?", "ETH balance"))
for i, acct in enumerate(ocn.accounts.list()):
    acct_balance = ocn.accounts.balance(acct)
    acct.password = manta_utils.user.password_map(acct.address, passwords)
    if acct.password:
        flg_password_exists = True
    else:
        flg_password_exists = False
    print("{:<5} {:<45} {:<20} {:<12} {}".format(i,acct.address, acct_balance.ocn, flg_password_exists, acct_balance.eth))

27 accounts exist
      Address                                       Ocean Token Balance  Password?    ETH balance
0     0x0207cb2f99EB2e005893d6108E2633641Ca9cC3e    0                    1            6092898583750000000
1     0x064789569D09b4d40b54383d84A25A840E5D67aD    4280                 1            99977130724000000000
2     0x06C0035fE67Cce2B8862D63Dc315D8C8c72207cA    100                  1            3000000000000000000
3     0x07aCfF76eB4A10c92B89965B0e174B35d887cB0b    2                    1            2999999994240380000
4     0x0A7a04DEdFE9A4859bcEdD2F21dcA8110499bed9    100                  1            3000000000000000000
5     0x1322A6ef2c560107733bFc622Fe556961Cb430a5    101                  1            2999999997919150000
6     0x1549193cc522A1716be4886879cD5f0964A3073C    100                  1            2999438222019160000
7     0x15C8249D657ad28daa56662ad4ee18dA4377d1a4    3                    1            3000000000000000000
8     0x170ff54d7bbD527B7bdF0477084

In [11]:
# Randomly select an account with a password
main_account = random.choice([acct for acct in ocn.accounts.list() if manta_utils.user.password_map(acct.address, passwords)])

### It is never secure to send your password over an unsecured HTTP connection, this is for demonstration only!
To interact with Ocean Protocol, use a wallet provider or the MetaMask browser extension.
See our documentation page for setting up your Ethereum accounts!

Most of your interaction with the Ocean Protocol blockchain smart contracts will require your Password.

## Requesting tokens
For development and testing, we have a magical function which will give you free testnet Ocean Token!

Your balance should be increased by 1 - but only after the block has been mined! Try printing your balance
multiple times until it updates.

In [12]:
print("Starting Ocean balance:", ocn.accounts.balance(main_account).ocn)
success = ocn.accounts.request_tokens(main_account, 1)
# The result will be true or false
assert success

Starting Ocean balance: 5


In [13]:
# Execute this after some time has passed to see the update!
print("Updated Ocean balance:", ocn.accounts.balance(main_account).ocn)

Updated Ocean balance: 6


## Asynchronous interactions
Many methods in the API will include a call to
[.waitForTransactionReceipt(transaction_hash)](https://web3py.readthedocs.io/en/stable/web3.eth.html#web3.eth.Eth.waitForTransactionReceiptj),
which explicitly pauses execution until the transaction has been mined. This will return the Transaction Receipt. When interacting
with the blockchain, things my take some time to execute!